In [ ]:
from __future__ import print_function
from __future__ import unicode_literals
import yaml
import json
import io
import procesamiento_microdatos as pm
import pandas as pd
import unicodecsv as csv
import openpyxl as pyxl
from collections import OrderedDict

In [ ]:
panel = pm.generate_panel()

In [ ]:
with open("grupos.yaml") as groups_file:
    grupos = yaml.load(groups_file)
    
with open("alias.yaml") as alias_file:
    alias = yaml.load(alias_file)
    
agrupamientos_usos = {
    "Otros Conceptos de Oferta": ['Variación de Stock', 'Búnker', 'No Aprovechado', 'Ajustes'],
    "Oferta Interna": ["Producción", "Importación", "Exportación", "Pérdidas", 'Variación de Stock', 'Búnker', 'No Aprovechado', 'Ajustes'],
    "Centrales Eléctricas": ['Servicio Público', 'Autoproducción'],
    "Otros Centros de Transformación": ['Aceiteras y Destilerías', 'Coquerías', 'Carboneras', 'Altos Hornos'],
    "Centros de Transformación": ['Servicio Público', 'Autoproducción', "Plantas de Tratamiento de Gas", "Refinerías", 'Aceiteras y Destilerías', 'Coquerías', 'Carboneras', 'Altos Hornos'],
    "Consumo Final": ["Consumo No Energético", "Residencial", "Transporte", "Comercial", "Industria", "Agropecuario"],
    "Consumo": ["Consumo Propio", "Consumo No Energético", "Residencial", "Transporte", "Comercial", "Industria", "Agropecuario"]
}

agrupamientos_energias = {
    "Energías Primarias": ["Energía Hidráulica", "Energía Nuclear", "Gas Natural de Pozo", "Petróleo", "Energía Eólica", "Energía Solar", "Carbón Mineral", "Alcohol Vegetal", "Aceites Vegetales", "Leña", "Bagazo", "Otros Primarios"],
    "Otras Fuentes Primarias de Energía": ["Energía Eólica", "Energía Solar", "Carbón Mineral", "Alcohol Vegetal", "Aceites Vegetales", "Leña", "Bagazo", "Otros Primarios"],
    "Aceites y Alcoholes Vegetales": ["Alcohol Vegetal", "Aceites Vegetales"],
    "Leña, Bagazo y Otros": ["Leña", "Bagazo", "Otros Primarios"],
    "Energías Secundarias":["Energía Eléctrica", "Gas Distribuido por Redes", "Gas Licuado", "Gas Oil", "Fuel Oil", "Motonaftas", "Biodiesel","Bioetanol", "Gas de Alto Horno", "No Energético de Carbón", "Carbón Vegetal", "Carbón Residual", "Coque de Petróleo", "Gas de Refinería", "Kerosene", "No Energético", "Otras Naftas", "Coque de Carbón", "Gas de Coquería",  "Etano", "Gasolina"],
    "Derivados del Petróleo": ["Gas Oil", "Fuel Oil", "Motonaftas"],
    "Biocombustibles": ["Biodiesel","Bioetanol"],
    "Otras Fuentes Secundarias de Energía": ["Gas de Alto Horno", "No Energético de Carbón", "Carbón Vegetal", "Carbón Residual", "Coque de Petróleo", "Gas de Refinería", "Kerosene", "No Energético", "Otras Naftas", "Coque de Carbón", "Gas de Coquería",  "Etano", "Gasolina"],
    "Energías Secundarias del Petróleo": ["Carbón Residual", "Coque de Petróleo", "Gas de Refinería", "Kerosene", "No Energético", "Otras Naftas"],
    "Energías Secundarias del Carbón": ["Coque de Carbón", "Gas de Coquería"],
    "Energías Secundarias del Gas": ["Etano", "Gasolina"]

}

In [ ]:
def unalias(df, alias=alias):
    """Usa el dict 'alias' para dar nombres completos al índice y las columnas de un DataFrame"""
    return df.rename(columns=alias, index=alias)

def calcular_perdidas(data):
    for uso in data.columns.get_values():
        if uso in agrupamientos_usos["Centros de Transformación"]:
            data.loc["Pérdidas por Transformación", uso] = -sum(data[uso].dropna())
        else:
            data.loc["Pérdidas por Transformación", uso] = 0
    return data

def get_yr(yr):
    """Devuelve la data correspondiente a un año del panel de microdatos con los nombres de energías y usos completos."""
    data = unalias(panel[yr])
    data = corregir_signo_consumo(data)
    data = calcular_perdidas(data)
    return data

def corregir_signo_consumo(df):
    # Corrijo signo de rubros de consumo para que "reciban" de las distintas formas de energía
    for consumo in ["Consumo No Energético", "Residencial", "Comercial", "Transporte", "Agropecuario", "Industria"]:
        df[consumo] = -df[consumo]
    return df

def make_links(yr):
    data = get_yr(yr)
    links = list()
    # Genero links directamente desde los microdatos cuando es posible
    for row in data.index.get_values():
        for col in data.columns.get_values():
            value = data.loc[row, col]
            if value < 0:
                links.append({"source": row, "target": col, "value": abs(value)})
            elif value > 0:
                links.append({"source": col, "target": row, "value": abs(value)})
    
    # Agrego los links desde las Centrales de Transformación a "Pérdidas por Transformación"
#    for centro in agrupamientos_usos["Centros de Transformación"]:
#        links.append({"source": centro,
#                      "target": "Pérdidas por Transformación",
#                      "value": abs(sum(data[centro]))
#                     })
    return links

def get_nodos(formato="dict"):
    wb = pyxl.load_workbook("maestro-nodos.xlsx")
    ws = wb.active
    raw_data = ws.values
    
    cols = next(raw_data)
    rows = list(raw_data)
    if formato == "dict":
        return [dict(zip(cols, row)) for row in rows]
    elif formato == "df":
        return pd.DataFrame.from_records(rows, columns=cols)
    else:
        print("Formato no reconodico: {}".format(formato))

In [ ]:
nodos = get_nodos("df")

In [ ]:
data2015 = get_yr(2015)

In [ ]:
links2015 = make_links(2015)

In [ ]:
data_energias = data2015.reset_index().rename_axis({"index": "Nombre"}, 1)

In [ ]:
data_energias = nodos.merge(data_energias)

In [ ]:
CAMPOS_DATAJSON = {
    "ID": "id",
    "Nombre": "name",
    "ID Padre": "parent",
    "Grupo": "group",
    "Posicion": "position"
}

def generar_lista_nodos(nodos):
    return nodos.drop(
        ["Eje", "Es Base","Nombre Padre"], axis=1
    ).rename(columns=CAMPOS_DATAJSON
    ).to_dict(orient='records')


In [ ]:
lista_de_nodos = generar_lista_nodos(nodos)

In [ ]:
def get_id_from_nombre(nombre_nodo):
    return [n for n in lista_de_nodos if n["name"] == nombre_nodo][0].get("id")

In [ ]:
def convert_names_to_ids(links_list):
    return([{"source": get_id_from_nombre(link["source"]),
             "target": get_id_from_nombre(link["target"]),
             "value": round(link["value"],2)}
            for link in links_list if round(link["value"],2)>=0.01])

In [ ]:
lista_de_links_con_id = convert_names_to_ids(links2015)

In [ ]:
datajson_2015 = {"nodes": lista_de_nodos, "links": lista_de_links_con_id}

In [ ]:
def write_json(obj, path):
    """Escribo un objeto a un archivo JSON con codificación UTF-8."""
    obj_str = json.dumps(obj, indent=4, separators=(",", ": "),
                         ensure_ascii=False)
    with io.open(path, "w", encoding='utf-8') as target:
        target.write(obj_str)


In [ ]:
nodos = get_nodos("df")
lista_de_nodos = generar_lista_nodos(nodos)
for i in panel.items:
    data = get_yr(i)
    links = make_links(i)
    lista_de_links_con_id = convert_names_to_ids(links)
    datajson = {"nodes": lista_de_nodos, "links": lista_de_links_con_id}
    write_json(datajson, "output/data_{}.json".format(i))

In [ ]:
panel[2015,:,:]

In [ ]:
def sumar_filas_df(df, nueva_fila, filas, borrar=True):
    """Suma todas las filas con índices en la lista `filas`,
    en una nueva fila con índice `nueva_fila`. Si `borrar`, no las incluye en el df retornado. Devuelve el df"""
    df.loc[nueva_fila] = reduce(pd.Series.add, [df.loc[f] for f in filas])
    if borrar:
        df = df.drop(filas, axis=0)
    return df
        
def adaptar_df_a_entidades_minem(df):
    for energia in agrupamientos_energias:
        df = sumar_filas_df(df, nueva_fila=energia, filas=agrupamientos_energias[energia])
    return df

def simplificar_usos(df):
    df_usos = df.transpose()
    for uso in agrupamientos_usos:
        df_usos = sumar_filas_df(df_usos, nueva_fila=uso, filas=agrupamientos_usos[uso])
    df = df_usos.transpose()
    return df

simplificar_usos(adaptar_df_a_entidades_minem(data2015))

In [ ]:
data2015.transpose()